# notebook to explore extracted movie features from neuroscout with pyns

In [33]:
import numpy as np
import pandas as pd

In [1]:
from pyns import Neuroscout
api = Neuroscout()

In [2]:
#there are 12 datasets
len(api.datasets.get())

12

In [3]:
datasets = api.datasets.get()
for i in datasets:
    print(i['name'], i['id'])

Budapest 27
NaturalisticNeuroimagingDatabase 28
HealthyBrainNetwork 8
SchematicNarrative 20
studyforrest 11
Raiders 10
Life 9
ParanoiaStory 18
Sherlock 21
SherlockMerlin 5
LearningTemporalStructure 19
ReadingBrainProject 29


In [4]:
budapest = datasets[0]

In [5]:
budapest['tasks']

[{'TR': 1.0,
  'avg_run_duration': 610,
  'id': 48,
  'n_runs_subject': 5,
  'n_subjects': 25,
  'name': 'movie',
  'summary': 'Movie watching'}]

### let's just look at budapest to start

In [6]:
api.runs.get(dataset_id=27, subject='sid000005',number=1)

[{'acquisition': None,
  'dataset_id': 27,
  'duration': 598.0,
  'id': 1433,
  'number': 1,
  'session': None,
  'subject': 'sid000005',
  'task': 48,
  'task_name': 'movie'}]

In [7]:
#get the run number for just BUDAPEST, SUBJECT5, PART1
run_id=api.runs.get(dataset_id=27, subject='sid000005',number=1)[0]['id']
run_duration=api.runs.get(dataset_id=27, subject='sid000005',number=1)[0]['duration']

In [8]:
[{'dataset_id': 27,
  'description': 'Average signal in CSF mask',
  'id': 37048,
  'max': 437.444,
  'mean': 369.345,
  'min': 287.63,
  'name': 'csf',
  'num_na': 0,
  'private': False,
  'source': 'fmriprep'},

SyntaxError: unexpected EOF while parsing (<ipython-input-8-2d0fea57a0a6>, line 10)

### get just the non-fmriprep predictors

In [68]:
budapest_predictors=api.predictors.get(run_id=run_id)
for i in budapest_predictors:
    if not i['source'] == 'fmriprep':
        print(i['name'],i['description'])

subtlexusfrequency_FREQcount Number of times the word appears in the corpus
subtlexusfrequency_FREQlow The number of times the word appears in the corpus starting with a lowercase letter
subtlexusfrequency_SUBTLWF The word frequency per million words
subtlexusfrequency_SUBTLCD Indicates in what percent of the films the word appears
subtlexusfrequency_Dom_PoS_SUBTLEX The dominant (most frequent) part of speech of each entry
subtlexusfrequency_Percentage_dom_PoS The relative frequency of the dominant part of speech
subtlexusfrequency_All_freqs_SUBTLEX The frequencies of each part of speech
affect_V.Mean.Sum Mean valence of spoken words.
affect_D.Mean.Sum Mean dominance of spoken words.
aoa_AoA_Kup Age of aquisition of spoken words.
massiveauditorylexicaldecision_NumPhones The number of phones in the item.
massiveauditorylexicaldecision_FreqCOCAspok Word frequency in the COCA corpus.spok
lancastersensorimotornorms_Auditory.mean how strongly the spoken word is experienced by hearing
lancas

#### get ids and names of predictors?

In [19]:
budapest_predictor_ids = []
budapest_predictor_names = []
for i in budapest_predictors:
    if not i['source'] == 'fmriprep':
        budapest_predictor_ids.append(i['id'])
        budapest_predictor_names.append(i['name'])

#### how many values are in each predictor??

In [99]:
predictor_lengths = []
for i in budapest_predictor_ids:
    predictor_lengths.append(
        len(api.predictor_events.get(predictor_id=i, run_id=run_id))
                                )

In [104]:
predictor_lengths=np.array(predictor_lengths)
print(predictor_lengths.min(), predictor_lengths.max(), predictor_lengths.mean())

10 5765 1529.017543859649


In [105]:
an_event=api.predictor_events.get(predictor_id=budapest_predictor_ids[37],run_id=run_id)

In [106]:
df = pd.DataFrame(columns=['onset','duration','value'])
for i in an_event:
    df = df.append({'onset': i['onset'], 'duration': i['duration'], 'value': i['value']}, ignore_index=True)

In [107]:
df

,onset,duration,value
0,10.0,1.00,0.016625665
1,11.0,1.00,0.016165715
2,12.0,1.00,0.02323487
3,13.0,1.00,0.011340556
4,14.0,1.00,0.0072452063
...,...,...,...
573,583.0,1.00,0.46360168
574,584.0,1.00,0.39699513
575,585.0,1.00,0.3872752
576,586.0,1.00,0.65442675


### given an event... convert it from duration onset value to timeseries
- sort it (the dicts are out of order)
- convert to timeseries
- resample it (eg some are variable mean 150 hz?? others are regular 0.6 hz)